# **The model parameters were obtained using Optuna.**
 
1. TfidfVectorizer seems to have only a little effect.As a code for comparison：
https://www.kaggle.com/code/jiaoyouzhang/wsdm-tfidf-voting-lgb-xgb-cat

3. I think simple classification models are not effective in solving this problem, and t he following perspectives should be considered:
    (1)Accuracy of content: Check if the answer accurately and correctly answers the question.
    (2)Integrity of content: Evaluate whether the answer comprehensively covers all aspects of the question.
    (3)Standardization of language: Evaluate whether the language expression of the answer is clear, accurate, and appropriate, and whether there are grammar or spelling errors.
    
    So,this code is for reference only. Please consider other more suitable methods.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_parquet("/kaggle/input/wsdm-cup-multilingual-chatbot-arena/train.parquet")
test = pd.read_parquet("/kaggle/input/wsdm-cup-multilingual-chatbot-arena/test.parquet")
submission=test.copy()


In [ ]:
######################################################################
train['prompt_len']=train['prompt'].apply(len)
train['response_a_len']=train['response_a'].apply(len)
train['response_b_len']=train['response_b'].apply(len)
train['response_a_len/prompt_len']=train['response_a_len']/train['prompt_len']
train['response_b_len/prompt_len']=train['response_b_len']/train['prompt_len']
train['response_b_len/response_a_len']=train['response_b_len']/train['response_a_len']

test['prompt_len']=test['prompt'].apply(len)
test['response_a_len']=test['response_a'].apply(len)
test['response_b_len']=test['response_b'].apply(len)
test['response_a_len/prompt_len']=test['response_a_len']/test['prompt_len']
test['response_b_len/prompt_len']=test['response_b_len']/test['prompt_len']
test['response_b_len/response_a_len']=test['response_b_len']/test['response_a_len']

train['winner'] = train['winner'].map({"model_a": 0, "model_b": 1})
test=test.drop(columns=['id','prompt','response_a','response_b','scored'], axis=1)
######################################################################

In [ ]:
X=train.drop(columns=['id','prompt','response_a','response_b','model_a','model_b','winner','language'], axis=1)
y=train['winner']


In [ ]:
lgb_param = {'num_leaves': 50, 'max_depth': 3, 'learning_rate': 0.06983803103008263, 'n_estimators': 275, 'reg_alpha': 0.01, 'reg_lambda': 51.776082079823745,
        'random_state': 42, 'verbose': -1}
xgb_param = {'n_estimators': 1509, 'learning_rate': 0.03, 'max_depth': 2, 'reg_lambda': 15.518731380893382, 'min_data_in_leaf': 2,
        'random_state': 42, 'verbose': -1}
cat_param = {'iterations': 207, 'learning_rate': 0.1, 'depth': 10, 'l2_leaf_reg': 22.370870995915176, 'min_data_in_leaf': 128,
        'random_state': 42, 'verbose': 0}

lgb_model = lgb.LGBMClassifier(**lgb_param)
xgb_model = xgb.XGBClassifier(**xgb_param)
cat_model = CatBoostClassifier(**cat_param)

In [ ]:
models = [('lgb', lgb_model), ('xgb', xgb_model), ('cat', cat_model)]
weights = [1, 1, 1] 
voting_clf = VotingClassifier(estimators=models, voting='soft', weights=weights)
voting_clf.fit(X, y)
y_pred = voting_clf.predict(test)

In [ ]:
y_pred_labels = ['model_a' if label == 0 else 'model_b' for label in y_pred]

In [ ]:
submission= pd.DataFrame({'id': submission['id'], 'winner': y_pred_labels})
submission.to_csv('submission.csv', index=False)